## Trần Ngọc Hân - 23520437

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Tải Dataset

In [2]:
%%bash
drive_dir="/content/drive/MyDrive/DL-Thực hành/mnist-dataset"
mkdir -p "$drive_dir"
cd "$drive_dir"

echo "Đang tải MNIST từ mirror an toàn..."

base_url="https://storage.googleapis.com/cvdf-datasets/mnist"

files=(
    "train-images-idx3-ubyte.gz"
    "train-labels-idx1-ubyte.gz"
    "t10k-images-idx3-ubyte.gz"
    "t10k-labels-idx1-ubyte.gz"
)

for file in "${files[@]}"; do
    echo "Đang tải $file ..."
    curl -L -o "$file" "$base_url/$file" --progress-bar
    gunzip -f "$file"
done

echo "Hoàn tất! File nằm trong $drive_dir"
ls -l


Đang tải MNIST từ mirror an toàn...
Đang tải train-images-idx3-ubyte.gz ...
Đang tải train-labels-idx1-ubyte.gz ...
Đang tải t10k-images-idx3-ubyte.gz ...
Đang tải t10k-labels-idx1-ubyte.gz ...
Hoàn tất! File nằm trong /content/drive/MyDrive/DL-Thực hành/mnist-dataset
total 53664
-rw------- 1 root root  7840016 Oct 17 03:33 t10k-images-idx3-ubyte
-rw------- 1 root root    10008 Oct 17 03:33 t10k-labels-idx1-ubyte
-rw------- 1 root root 47040016 Oct 17 03:33 train-images-idx3-ubyte
-rw------- 1 root root    60008 Oct 17 03:33 train-labels-idx1-ubyte


######################################################################## 100.0%
######################################################################## 100.0%
######################################################################## 100.0%
######################################################################## 100.0%


## Tạo các file cần thiết

In [3]:
%%bash
base_dir="/content/drive/MyDrive/DL-Thực hành/"
touch "$base_dir/main.py"
touch "$base_dir/perceptron_1_layer.py"
touch "$base_dir/perceptron_3_layer.py"
touch "$base_dir/mnist_dataset.py"
touch "$base_dir/main.py"

echo "Đã tạo 4 file rỗng trong $base_dir:"
ls -l "$base_dir"

Đã tạo 4 file rỗng trong /content/drive/MyDrive/DL-Thực hành/:
total 1112
-rw------- 1 root root 112545 Oct  3 02:41 Bai-Thuc-Hanh-1.ipynb
-rw------- 1 root root  33253 Oct 17 03:29 checkpoint_perceptron_1_layer.pth
-rw------- 1 root root 943253 Oct 17 03:30 checkpoint_perceptron_3_layer.pth
-rw------- 1 root root   4219 Oct 17 03:33 main.py
drwx------ 2 root root   4096 Oct 17 03:33 mnist-dataset
-rw------- 1 root root    949 Oct 17 03:33 mnist_dataset.py
-rw------- 1 root root  33529 Oct 17 03:32 notebook.ipynb
-rw------- 1 root root    472 Oct 17 03:33 perceptron_1_layer.py
-rw------- 1 root root    728 Oct 17 03:33 perceptron_3_layer.py
drwx------ 2 root root   4096 Oct 17 03:28 __pycache__


In [ ]:
!pip install idx2numpy

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 18, in <module>
    from pip._internal.index.collector import LinkCollector
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/index/collector.py", line 31, in <module>
    from pip._vendor import requests
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/requests/__init__.py", line 43, in <module>
    from pip._vendor import urllib3
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/urllib3/__init__.py", line 13, in <module>
    from .connectionpool import HTTPConnectionPool, HTTPSConnectionPool, connection_from_url
  File "/usr/local/lib/py

### mnist_dataset.py

In [4]:
%%writefile "/content/drive/MyDrive/DL-Thực hành/mnist_dataset.py"
import torch
from torch.utils.data import Dataset
import idx2numpy
import numpy as np

def collate_fn(items: list[dict]) -> dict:
    items = [{
        "image": np.expand_dims(item["image"], axis=0),
        "label": np.array(item["label"])
    } for item in items]

    items = {
        "image": np.stack([item["image"] for item in items], axis=0),
        "label": np.stack([item["label"] for item in items], axis=0)
    }
    return items


class MnistDataset(Dataset):
    def __init__(self, image_path: str, label_path: str):
        images = idx2numpy.convert_from_file(image_path)
        labels = idx2numpy.convert_from_file(label_path)

        self._data = [{
            "image": image.astype(np.float32) / 255.0,
            "label": int(label)
        } for image, label in zip(images, labels)]

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, idx: int) -> dict:
        return self._data[idx]

Overwriting /content/drive/MyDrive/DL-Thực hành/mnist_dataset.py


## Bai 1 - PERCEPTRON_1_LAYER

In [5]:
%%writefile "/content/drive/MyDrive/DL-Thực hành/perceptron_1_layer.py"
import torch
from torch import nn
from torch.nn import functional as F

class Perceptron_1_layer(nn.Module):
    def __init__(self, image_size=(28,28), num_labels=10):
        super().__init__()
        w, h = image_size
        self.linear = nn.Linear(in_features=w*h, out_features=num_labels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        x = self.linear(x)
        return F.log_softmax(x, dim=1)

Overwriting /content/drive/MyDrive/DL-Thực hành/perceptron_1_layer.py


## Bai 2 - PERCEPTRON_3_LAYER

In [6]:
%%writefile "/content/drive/MyDrive/DL-Thực hành/perceptron_3_layer.py"
import torch
import torch.nn as nn
import torch.nn.functional as F


class Perceptron_3_layer(nn.Module):
    def __init__(self, input_dim=28*28, hidden1=256, hidden2=128, num_classes=10):
        super(Perceptron_3_layer, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, num_classes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(x.size(0), -1)

        # Hidden layer 1
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # Hidden layer 2
        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        # Output layer
        out = self.fc3(x)
        return out

Overwriting /content/drive/MyDrive/DL-Thực hành/perceptron_3_layer.py


## KẾT QUẢ

In [7]:
%%writefile "/content/drive/MyDrive/DL-Thực hành/main.py"
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from mnist_dataset import collate_fn, MnistDataset
from perceptron_1_layer import Perceptron_1_layer
from perceptron_3_layer import Perceptron_3_layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = []
    for batch in dataloader:
        X = torch.tensor(batch["image"], dtype=torch.float32).to(device)
        y = torch.tensor(batch["label"], dtype=torch.long).to(device)

        optimizer.zero_grad()
        output = model(X)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    return losses


def evaluate(dataloader, model):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            X = torch.tensor(batch["image"], dtype=torch.float32).to(device)
            y = torch.tensor(batch["label"], dtype=torch.long).to(device)

            output = model(X)
            preds = torch.argmax(output, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds) * 100
    precision = precision_score(all_labels, all_preds, average='macro') * 100
    recall = recall_score(all_labels, all_preds, average='macro') * 100
    f1 = f1_score(all_labels, all_preds, average='macro') * 100

    print("\nThống kê kết quả:")
    print(classification_report(all_labels, all_preds, digits=4))

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


if __name__ == "__main__":
    model_type = input("Chọn mô hình (1layer / 3layer): ").strip().lower()

    # load data
    train_dataset = MnistDataset(
        image_path="/content/drive/MyDrive/DL-Thực hành/mnist-dataset/train-images-idx3-ubyte",
        label_path="/content/drive/MyDrive/DL-Thực hành/mnist-dataset/train-labels-idx1-ubyte"
    )
    test_dataset = MnistDataset(
        image_path="/content/drive/MyDrive/DL-Thực hành/mnist-dataset/t10k-images-idx3-ubyte",
        label_path="/content/drive/MyDrive/DL-Thực hành/mnist-dataset/t10k-labels-idx1-ubyte"
    )

    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

    # khởi tạo
    if model_type == "1layer":
        model = Perceptron_1_layer(image_size=(28, 28), num_labels=10).to(device)
        checkpoint_path = "/content/drive/MyDrive/DL-Thực hành/checkpoint_perceptron_1_layer.pth"
    elif model_type == "3layer":
        model = Perceptron_3_layer(input_dim=28*28, num_classes=10).to(device)
        checkpoint_path = "/content/drive/MyDrive/DL-Thực hành/checkpoint_perceptron_3_layer.pth"
    else:
        raise ValueError("Chỉ được chọn '1layer' hoặc '3layer'.")

    print(f"\nĐang huấn luyện mô hình: {model_type}\n")

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    EPOCHS = 5
    best_score = 0
    best_score_name = "f1"
    history_loss = []

    # train loop
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        losses = train(train_dataloader, model, loss_fn, optimizer)
        avg_loss = np.mean(losses)
        history_loss.append(avg_loss)
        print(f"Loss trung bình: {avg_loss:.4f}")

        scores = evaluate(test_dataloader, model)
        for score_name in scores:
            print(f"\t- {score_name}: {scores[score_name]:.2f}")

        current_score = scores[best_score_name]
        if current_score > best_score:
            best_score = current_score
            torch.save(model.state_dict(), checkpoint_path)
            print("Lưu mô hình tốt nhất.")

    print(f"\nHuấn luyện xong mô hình {model_type}. Best F1: {best_score:.2f}%")

Overwriting /content/drive/MyDrive/DL-Thực hành/main.py


### Kết quả perceptron 1 layer

In [8]:
!python3 "/content/drive/MyDrive/DL-Thực hành/main.py"

Chọn mô hình (1layer / 3layer): 1layer

Đang huấn luyện mô hình: 1layer


Epoch 1/5
Loss trung bình: 0.9884

Thống kê kết quả:
              precision    recall  f1-score   support

           0     0.9188    0.9582    0.9381       980
           1     0.8928    0.9612    0.9258      1135
           2     0.8705    0.8207    0.8449      1032
           3     0.8367    0.8673    0.8517      1010
           4     0.8506    0.8870    0.8684       982
           5     0.8645    0.7298    0.7915       892
           6     0.8785    0.9134    0.8956       958
           7     0.8875    0.8667    0.8770      1028
           8     0.8217    0.8234    0.8226       974
           9     0.8444    0.8226    0.8333      1009

    accuracy                         0.8673     10000
   macro avg     0.8666    0.8650    0.8649     10000
weighted avg     0.8670    0.8673    0.8662     10000

	- accuracy: 86.73
	- precision: 86.66
	- recall: 86.50
	- f1: 86.49
Lưu mô hình tốt nhất.

Epoch 2/5
Loss trung b

### Kết quả perceptron 3 layer

In [ ]:
!python3 "/content/drive/MyDrive/DL-Thực hành/main.py"

Chọn mô hình (1layer / 3layer): 3layer

Đang huấn luyện mô hình: 3layer


Epoch 1/5
Loss trung bình: 1.7836

Thống kê kết quả:
              precision    recall  f1-score   support

           0     0.8747    0.9612    0.9159       980
           1     0.8449    0.9744    0.9051      1135
           2     0.8108    0.8178    0.8143      1032
           3     0.6160    0.8861    0.7268      1010
           4     0.6808    0.8493    0.7558       982
           5     0.8734    0.3094    0.4570       892
           6     0.8450    0.9050    0.8740       958
           7     0.7776    0.8842    0.8275      1028
           8     0.8343    0.5791    0.6836       974
           9     0.7853    0.5510    0.6476      1009

    accuracy                         0.7793     10000
   macro avg     0.7943    0.7718    0.7607     10000
weighted avg     0.7937    0.7793    0.7656     10000

	- accuracy: 77.93
	- precision: 79.43
	- recall: 77.18
	- f1: 76.07
Lưu mô hình tốt nhất.

Epoch 2/5
Loss trung b